In [ ]:
!nvidia-smi

Thu Sep  8 02:01:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#import os
#os.chdir('C:/Users/star9/Downloads')
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
heaven = pd.read_excel('/content/gdrive/MyDrive/bert/datacamp_4.xlsx')
heaven.head()

,년도,차수,과정명,sentence,분류,sent,score
0,2022.0,4.0,(DX실무01) LG Data 분석 Camp,천천히 알려주셔서 이애하기 편했습니다.,NaN,LABEL_1,0.971641
1,2022.0,4.0,(DX실무01) LG Data 분석 Camp,마지막 튜닝 부분에서 급하게 설명이 진행되어 약간 헷갈리는 부분이 있습니다.,NaN,LABEL_0,0.081353
2,2022.0,4.0,(DX실무01) LG Data 분석 Camp,전체 흐름을 알려주셔서 좋았습니다.,NaN,LABEL_1,0.973427
3,2022.0,4.0,(DX실무01) LG Data 분석 Camp,2주간의 시간동안 천천히 배울 수 있었던 점 - 전반적으로 좋음,NaN,LABEL_1,0.971072
4,2022.0,4.0,(DX실무01) LG Data 분석 Camp,기존 두루뭉술하게 알고 있던 파이썬 활용 데이터 분석을 체계적으로 알 수 있었다.,NaN,LABEL_1,0.939500


In [ ]:
heaven.head()

,index,년도,구분,과정명,차수,sentence,긍정,부정,내용구성,관심유발,자기개발,기타
0,0,2017.0,경영교육센터,EnDP 101,1.0,2일 정도의 일정으로 해도 될 듯함,0,1.0,1,0,0,1.0
1,1,2017.0,경영교육센터,EnDP 102,1.0,일반 교육사간이 매우 적당하였습니다,1.0,0.0,1,0,0,1.0
2,2,2017.0,경영교육센터,EnDP 101,1.0,프로그램도 생각보다 매우 실제적이고 구체적으로 도움이 되었습니다,1.0,0.0,1,0,1,0.0
3,3,2017.0,경영교육센터,EnDP 101,1.0,특히 본인이 교육전 미리 보완하고 싶은 교육과제협업를 통해 문제의식과 전문강사로부터...,1.0,0.0,1,0,1,0.0
4,4,2017.0,경영교육센터,EnDP 101,1.0,인화원진행자분들께도 진심으로 수고하셨다는 감사말씀을 전합니다,1.0,0.0,0,0,0,1.0


In [ ]:
#heaven = heaven.drop(9052) 라벨링 시 잘못 마크된 행 제거
heaven.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77970 entries, 0 to 77969
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        77970 non-null  int64  
 1   년도           77970 non-null  object 
 2   구분           77970 non-null  object 
 3   과정명          77970 non-null  object 
 4   차수           77970 non-null  float64
 5   sentence     77970 non-null  object 
 6   긍정           77970 non-null  object 
 7   부정           77970 non-null  float64
 8   내용구성         77970 non-null  int64  
 9   관심유발         77970 non-null  int64  
 10  자기개발         77970 non-null  int64  
 11  기타           77970 non-null  object 
 12  Unnamed: 11  77970 non-null  float64
 13  Unnamed: 12  77970 non-null  object 
 14  pred         77970 non-null  int64  
 15  changed      77970 non-null  int64  
dtypes: float64(3), int64(6), object(7)
memory usage: 9.5+ MB


In [ ]:
heaven['자기개발'] = heaven['자기개발'].astype(int)
heaven['관심유발'] = heaven['관심유발'].astype(int)
heaven['내용구성'] = heaven['내용구성'].astype(int)

In [ ]:
heaven = heaven.reset_index()

In [ ]:
#########################################bert###################################################
################################################################################################
! pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-gigd4wyt
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-gigd4wyt


In [ ]:
! pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#타겟들이 float로 되어있어서 인식 못함 -> int로 바꿈
#heaven['관심유발'] = heaven['관심유발'].astype(int)

In [ ]:
data_list = []
for q, label in zip(heaven['sentence'], heaven['내용구성'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 32
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#토크나이징이 잘 됐는지 확인
len(data_train)

62376

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#########################요 셀만 돌아가면 됨#######################################################
##################################################################################################

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

NameError: ignored

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1950 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7241703271865845 train acc 0.5625
epoch 1 batch id 201 loss 0.6557952165603638 train acc 0.5402674129353234
epoch 1 batch id 401 loss 0.5455952882766724 train acc 0.6054395261845387
epoch 1 batch id 601 loss 0.7514187693595886 train acc 0.6369072379367721
epoch 1 batch id 801 loss 0.6885945796966553 train acc 0.6544943820224719
epoch 1 batch id 1001 loss 0.4699738621711731 train acc 0.6646478521478522
epoch 1 batch id 1201 loss 0.5336399078369141 train acc 0.6726946294754371
epoch 1 batch id 1401 loss 0.6228069067001343 train acc 0.6771056388294076
epoch 1 batch id 1601 loss 0.4789105951786041 train acc 0.6818980324797002
epoch 1 batch id 1801 loss 0.5541688799858093 train acc 0.6854525263742365
epoch 1 train acc 0.6882371794871794


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/488 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7307377049180328


  0%|          | 0/1950 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.24166762828826904 train acc 0.9375
epoch 5 batch id 201 loss 0.24572670459747314 train acc 0.8722014925373134
epoch 5 batch id 401 loss 0.19244617223739624 train acc 0.8756234413965087
epoch 5 batch id 601 loss 0.7566697001457214 train acc 0.8767158901830283
epoch 5 batch id 801 loss 0.4577108323574066 train acc 0.8775749063670412
epoch 5 batch id 1001 loss 0.15086887776851654 train acc 0.8798076923076923
epoch 5 batch id 1201 loss 0.271305114030838 train acc 0.881426935886761
epoch 5 batch id 1401 loss 0.39996203780174255 train acc 0.8799518201284796
epoch 5 batch id 1601 loss 0.17413781583309174 train acc 0.8805824484697065
epoch 5 batch id 1801 loss 0.25690966844558716 train acc 0.8800666296501943
epoch 5 train acc 0.8801121794871795


  0%|          | 0/488 [00:00<?, ?it/s]

epoch 5 test acc 0.7326588114754099


In [ ]:
#모델 저장하기

torch.save(model.state_dict(), '/content/gdrive/MyDrive/bert/paramcontent.pt')
torch.save(model, '/content/gdrive/MyDrive/bert/modelcontent.pt')

In [ ]:
#모델 불러오기

model = torch.load('/content/gdrive/MyDrive/bert/modelcontent.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:

#test

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                return(0)
            elif np.argmax(logits) == 1:
                return(1)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#새로운 데이터 불러와 테스트
hr = pd.read_excel('/content/gdrive/MyDrive/hr.xlsx')
hr.head()

,sentence,소속사,성별,직무,근속 년수,직급
0,로봇공학 박사님 강의가 재밌었습니다,LG화학,남,HRM,5년~8년,선임
1,강의시간이 좀 짧은 느낌이었습니다,LG화학,남,HRM,5년~8년,선임
2,데니스홍교수님의 로봇관련 세미나,LG전자,남,HR외 직군,9년 이상,책임
3,원하는 과정을 선텍적으로 참여할수 있도록 구성해서 좋았습니다,LG전자,남,HR외 직군,9년 이상,책임
4,제일 마지막 교육이였던 뇌 심리 교육이 인상적이였습니다,기타,남,HRM,9년 이상,책임


In [ ]:
from tqdm import tqdm
tqdm.pandas()
heaven['내용구성'] = heaven['sentence'].progress_apply(predict)

100%|██████████| 53/53 [00:18<00:00,  2.94it/s]


In [ ]:
from tqdm import tqdm
tqdm.pandas()
heaven['pred'] = heaven['sentence'].progress_apply(predict)

100%|██████████| 77970/77970 [17:18:28<00:00,  1.25it/s]


In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(heaven['내용구성'], heaven['pred'])
print(cf)

[[38971 30742]
 [ 6135  2122]]


In [ ]:
heaven['내용구성'].value_counts()

0    53
Name: 내용구성, dtype: int64

In [ ]:
heaven['changed'] = abs(heaven['내용구성'] - heaven['pred'])

In [ ]:
heaven.to_csv('/content/gdrive/MyDrive/bert/bert내용.csv', encoding='utf-8-sig')
#현업적용 915 - 12% #관심유발 1560 - 20% #내용구성 - 47% #긍정 -66%

In [ ]:
heaven.to_csv('/content/gdrive/MyDrive/bert/4차수.csv', encoding = 'utf-8-sig', index=None)